In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
from functools import reduce

In [2]:
spark = SparkSession.builder.appName("MeuApp").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/12 15:08:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:

df_classe = spark.read.option("delimiter", ",").option("header", True).csv("classe.csv")

df_classe.show(truncate=False)

+-----------+-------------+-------+
|classe     |codigo_regiao|posicao|
+-----------+-------------+-------+
|CLASSE 21_5|21060        |5      |
|CLASSE 98_5|98495        |5      |
|CLASSE 38_5|38321        |5      |
|CLASSE 68_5|68789        |5      |
|CLASSE 72_5|71236        |5      |
|CLASSE 24_5|24147        |5      |
|CLASSE 07_5|07663        |5      |
|CLASSE 05_5|05886        |5      |
|CLASSE 77_5|77370        |5      |
|CLASSE 98_5|98574        |5      |
|CLASSE 04_5|04907        |5      |
|CLASSE 19_5|19892        |5      |
|CLASSE 33_5|33513        |5      |
|CLASSE 07_2|07           |2      |
|CLASSE 05_2|05           |2      |
|CLASSE 77_2|77           |2      |
|CLASSE 33_2|33           |2      |
|CLASSE 0_1 |0            |1      |
+-----------+-------------+-------+



In [4]:

df_fake = spark.read.option("delimiter", ",").option("header", True).csv("data.csv")\
.withColumn("data_nascimento", F.to_date(F.col("data_nascimento")))\
.withColumn("cep", F.regexp_replace(F.col("cep"), "-", ""))\
.withColumn("p_1", F.substring(F.col("cep"),  1, 1))\
.withColumn("p_2", F.substring(F.col("cep"),  1, 2))\
.withColumn("p_3", F.substring(F.col("cep"),  1, 3))\
.withColumn("p_4", F.substring(F.col("cep"),  1, 4))\
.withColumn("p_5", F.substring(F.col("cep"),  1, 5))\

df_fake.show(truncate=False)

+------+------------------------+----------------------------------+---------------+--------+---+---+---+----+-----+
|id    |nome                    |email                             |data_nascimento|cep     |p_1|p_2|p_3|p_4 |p_5  |
+------+------------------------+----------------------------------+---------------+--------+---+---+---+----+-----+
|605755|Catarina Rios           |costelasara@example.com           |1941-07-19     |72828003|7  |72 |728|7282|72828|
|453942|João Gabriel Nunes      |arthurdas-neves@example.org       |2004-12-01     |05055581|0  |05 |050|0505|05055|
|264157|Sofia Araújo            |bda-costa@example.com             |1910-11-02     |30758720|3  |30 |307|3075|30758|
|284311|Alexia Fogaça           |machadonina@example.org           |1929-01-29     |28840655|2  |28 |288|2884|28840|
|640344|Srta. Alexia Fonseca    |joao-guilhermesousa@example.org   |2002-11-26     |85644776|8  |85 |856|8564|85644|
|566589|Esther Correia          |vargasmateus@example.com       

In [5]:
F

<module 'pyspark.sql.functions' from '/root/hadoop/spark-3.5.1/python/pyspark/sql/functions.py'>

In [6]:
df_fake_p1=df_fake.join(df_classe.alias("df_classe_1"), how='left', on=(df_fake.p_1==F.col("df_classe_1.codigo_regiao") )).select(df_fake.columns+[F.col("df_classe_1.classe").alias("classe_p1")])
df_fake_p2=df_fake_p1.join(df_classe.alias("df_classe_2"), how='left', on=(df_fake.p_2==F.col("df_classe_2.codigo_regiao"))).select(df_fake_p1.columns+[F.col("df_classe_2.classe").alias("classe_p2")])
df_fake_p3=df_fake_p2.join(df_classe.alias("df_classe_3"), how='left', on=(df_fake.p_3==F.col("df_classe_3.codigo_regiao"))).select(df_fake_p2.columns+[F.col("df_classe_3.classe").alias("classe_p3")])
df_fake_p4=df_fake_p3.join(df_classe.alias("df_classe_4"), how='left', on=(df_fake.p_4==F.col("df_classe_4.codigo_regiao"))).select(df_fake_p3.columns+[F.col("df_classe_4.classe").alias("classe_p4")])
df_fake_p5=df_fake_p4.join(df_classe.alias("df_classe_5"), how='left', on=(df_fake.p_5==F.col("df_classe_5.codigo_regiao"))).select(df_fake_p4.columns+[F.col("df_classe_5.classe").alias("classe_p5")])

In [7]:
df_result = df_fake_p5.select(df_fake.columns+[F.expr("coalesce(classe_p5,classe_p4,classe_p3,classe_p2,classe_p1)").alias("classe")])

In [11]:
df_result.filter(F.col("classe").isNotNull()).orderBy("p_5", ascending=False).show(truncate=False)

+------+---------------------+----------------------------------+---------------+--------+---+---+---+----+-----+-----------+
|id    |nome                 |email                             |data_nascimento|cep     |p_1|p_2|p_3|p_4 |p_5  |classe     |
+------+---------------------+----------------------------------+---------------+--------+---+---+---+----+-----+-----------+
|812112|Benjamim Sampaio     |giovanna98@example.org            |1915-01-24     |77613170|7  |77 |776|7761|77613|CLASSE 77_2|
|834388|Bella Melo           |cavalcantevitor@example.com       |2012-02-24     |77192882|7  |77 |771|7719|77192|CLASSE 77_2|
|660018|Arthur Pimenta       |daniela24@example.org             |1940-08-18     |77030923|7  |77 |770|7703|77030|CLASSE 77_2|
|409883|Sra. Brenda Pimenta  |bryan25@example.com               |1942-06-27     |33971818|3  |33 |339|3397|33971|CLASSE 33_2|
|434663|Maria Fernanda Campos|fernandesvitor-gabriel@example.org|1942-09-08     |33513339|3  |33 |335|3351|33513|CLASS

In [9]:
# df_report = df.groupBy("posicao").count()
# df_report = df_report.orderBy("posicao", ascending=False)
# df_report.show(truncate=False)